<a href="https://colab.research.google.com/github/nadiarvi/Sentiment-Analysis-using-RNN-and-LSTM/blob/main/Sentiment_Analysis_using_RNN_and_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objectives
* Design an LSTM-based text classification model from scratch using PyTorch.
* Apply the classification model to a popular classification task, sentiment analysis.
* Use Huggingface Library (`transformers`) to leverage self-supervision via large language models.

## Download Train Dataset

In [1]:
import os
import sys
import ssl
import urllib.request

In [2]:
def _download_dataset(size=10000):
    assert sys.version_info >= (3, 6), "Use Python3.6+"

    url = f"https://raw.githubusercontent.com/dongkwan-kim/small_dataset/master/review_{size // 1000}k.csv"

    dir_path = "../data"
    file_path = os.path.join(dir_path, f"review_{size // 1000}k.csv")
    if not os.path.isfile(file_path):
        print(f"Download: {file_path}")
        os.makedirs(dir_path, exist_ok=True)
        ctx = ssl._create_unverified_context()
        with urllib.request.urlopen(url, context=ctx) as u, open(file_path, "wb") as f:
            f.write(u.read())
    else:
        print(f"Already exist: {file_path}")

In [3]:
_download_dataset()

Download: ../data/review_10k.csv


## Environment Setting

In [4]:
! pip install transformers[torch] datasets evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [5]:
import time
import random

from tqdm.auto import tqdm
from easydict import EasyDict

import torch
import torch.nn as nn
import torch.optim as optim

from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset

In [6]:
def seed_reset(seed=0):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [7]:
seed_reset(42)

# LSTM

## Utils for Text Classification using LSTM

In [8]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [9]:
def accuracy(prediction, label):
    binary_prediction = torch.round(torch.sigmoid(prediction))
    correct = (binary_prediction == label).float()
    acc = correct.sum() / len(correct)
    return acc

In [10]:
def _load_dataset_for_LSTM(batch_size, num_samples=10000, train_test_ratio=0.8, seed=42, test_data_path=None):
    dataset = load_dataset("csv", data_files=f"../data/review_{num_samples // 1000}k.csv", split="train")
    dataset = dataset.train_test_split(test_size=1 - train_test_ratio, seed=seed)
    train_dataset, validation_dataset = dataset["train"], dataset["test"]

    tokenizer = get_tokenizer("basic_english")

    # Build vocabulary
    def yield_tokens(data_iter):
        for sample in data_iter:
            yield tokenizer(sample["review"])

    vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>", "<pad>"])
    vocab.set_default_index(vocab["<unk>"])

    # Tokenize & Build data loader
    def data_collator(batch):
        texts, text_lengths, sentiments = [], [], []
        for sample in batch:
            tokenized_text = vocab(tokenizer(sample['review']))
            texts.append(tokenized_text)
            text_lengths.append(len(tokenized_text))
            sentiments.append(sample['sentiment'])

        text_batch = torch.zeros(len(batch), max(text_lengths), dtype=torch.int64)
        length_batch, sentiment_batch = [], []
        for i, (text, length, sentiment) in enumerate(sorted(zip(texts, text_lengths, sentiments), key=lambda x: x[1], reverse=True)):
            for j, word in enumerate(text):
                text_batch[i][j] = text[j]
            length_batch.append(length)
            sentiment_batch.append(sentiment)

        batch = EasyDict()
        batch.review = (text_batch, length_batch)
        batch.sentiment = torch.tensor(sentiment_batch, dtype=torch.float64)
        return batch

    train_dataloader = DataLoader(
        train_dataset, shuffle=False, collate_fn=data_collator, batch_size=batch_size
    )
    validation_dataloader = DataLoader(
        validation_dataset, shuffle=False, collate_fn=data_collator, batch_size=batch_size
    )

    test_dataloader = None
    if test_data_path != None:
        test_dataset = load_dataset('csv', data_files=test_data_path, split='train')
        test_dataloader = DataLoader(
            test_dataset, shuffle=False, collate_fn=data_collator, batch_size=batch_size
        )

    return train_dataloader, validation_dataloader, test_dataloader, len(vocab), vocab(["<pad>"])[0]

## Implement LSTM with PyTorch

In [11]:
class RNN(nn.Module):

    def __init__(self, num_embeddings, padding_idx, embedding_dim,
                 hidden_dim, num_layers, dropout, bidirectional):
        """
        :param num_embeddings: the numebr of embeddings (vocab size)
        :param padding_idx: padding idx
        :param embedding_dim: (int) embedding dimension
        :param hidden_dim: (int) hidden dimension
        :param num_layers: (int) the number of recurrent layers
        :param dropout: (float) dropout rate
        :param bidirectional: (bool) is bidirectional

        :return output: type=torch.Tensor, shape=[batch size]
        """

        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
        self.dropout = nn.Dropout(dropout)

        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

        out = hidden_dim * 2 if bidirectional else hidden_dim
        self.fc = nn.Linear(out, 1)


    def forward(self, text, text_lengths):

        embedded = self.dropout(self.embedding(text))

        packed_embedded = pack_padded_sequence(embedded, text_lengths, batch_first=True)

        output, (h_n, c_n) = self.rnn(packed_embedded)

        sec_last_layer = h_n[-2,:,:]
        last_layer = h_n[-1,:,:]

        if self.rnn.bidirectional:
        # Concatenate forward and backward if bidirectional
          h_n = torch.cat((sec_last_layer, last_layer), dim=1)
        else:
          h_n = last_layer

        output = self.fc(h_n).squeeze(-1)


        assert output.shape == torch.Size([text.shape[0]]) # batch_size
        return output

In [12]:
def train(model, iterator, optimizer, criterion, device):
    """ Complete train method
    :param model: RNN model
    :param iterator: train dataset iterator
    :param optimizer: optimzer
    :param criterion: loss function

    :return output: train loss, train accuracy
    """

    total_epoch_loss = 0
    total_epoch_acc = 0

    model.train()

    for batch in tqdm(iterator, desc="train"):

        optimizer.zero_grad()
        (text, text_lengths), labels = batch.review, batch.sentiment
        text = text.to(device)
        labels = labels.to(device)

        # prediction
        predictions = model(text, text_lengths).squeeze(-1)

        # loss
        loss = criterion(predictions, labels.float())

        # accuracy
        acc = accuracy(predictions, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        assert loss.shape == torch.Size([])
        assert acc.shape == torch.Size([])

        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()


    epoch_loss = total_epoch_loss / len(iterator)
    epoch_acc = total_epoch_acc / len(iterator)
    return epoch_loss, epoch_acc


In [13]:
def evaluate(model, iterator, criterion, device):
    """ Complete evaluate method
    :param model: RNN model
    :param iterator: dataset iterator
    :param criterion: loss function

    :return output: loss, accuracy
    """

    total_epoch_loss = 0
    total_epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for batch in tqdm(iterator, desc="evaluate"):
            (text, text_lengths), labels = batch.review, batch.sentiment
            text = text.to(device)
            labels = labels.to(device)

            # Forward pass
            predictions = model(text, text_lengths).squeeze(-1)

            # loss
            loss = criterion(predictions, labels.float())

            # accuracy
            acc = accuracy(predictions, labels)

            assert loss.shape == torch.Size([])
            assert acc.shape == torch.Size([])

            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    epoch_loss = total_epoch_loss / len(iterator)
    epoch_acc = total_epoch_acc / len(iterator)
    return epoch_loss, epoch_acc

In [14]:
def run_LSTM(hyperparameter_dict, num_samples=10000, train=train, evaluate=evaluate, seed=0, test_data_path=None, verbose=True):
    """
    You do not have to consider test_data_path, since it will be used for grading only.
    You can modify this run function for training your own model in the marked area below.
    """
    train_loader, valid_loader, test_loader, vocab_size, padding_idx = _load_dataset_for_LSTM(hyperparameter_dict.batch_size, num_samples, seed=seed, test_data_path=test_data_path)

    NUM_EMBEDDINGS = vocab_size
    PADDING_IDX = padding_idx

    model = RNN(NUM_EMBEDDINGS,
                PADDING_IDX,
                hyperparameter_dict.embedding_dim,
                hyperparameter_dict.hidden_dim,
                hyperparameter_dict.num_layers,
                hyperparameter_dict.dropout,
                hyperparameter_dict.bidirectional
                )

    device = torch.device(hyperparameter_dict.device if torch.cuda.is_available() else 'cpu')

    train_loss, train_acc = None, None
    valid_loss, valid_acc = None, None
    test_loss, test_acc = None, None

    optimizer = optim.Adam(model.parameters(), lr=hyperparameter_dict.learning_rate)
    criterion = nn.BCEWithLogitsLoss()
    model = model.to(device)
    criterion = criterion.to(device)

    for epoch in range(hyperparameter_dict.num_epochs):
        print(f'Epoch: {epoch+1:02}')
        start_time = time.time()

        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        valid_loss, valid_acc = evaluate(model, valid_loader, criterion, device)

        if verbose:
            end_time = time.time()
            epoch_mins, epoch_secs = epoch_time(start_time, end_time)
            print(f'\nEpoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    if test_loader != None:
        test_loss, test_acc = evaluate(model, test_loader, criterion)
        print(f'\t Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

    return train_loss, train_acc, valid_loss, valid_acc, test_loss, test_acc


## Run LSTM

In [15]:
def set_hyperparameter_for_LSTM():
    """
    Set your best hyperparameters for your model
    """
    param_dict = EasyDict()
    param_dict.embedding_dim = 128
    param_dict.hidden_dim = 64
    param_dict.num_layers = 2
    param_dict.dropout = 0.4
    param_dict.bidirectional = True
    param_dict.batch_size = 64
    param_dict.num_epochs = 10
    param_dict.learning_rate = 1e-3
    param_dict.device = 'cuda'
    return param_dict

In [16]:
param_dict = set_hyperparameter_for_LSTM()
run_LSTM(param_dict)

Generating train split: 0 examples [00:00, ? examples/s]

Epoch: 01


train:   0%|          | 0/125 [00:00<?, ?it/s]

evaluate:   0%|          | 0/32 [00:00<?, ?it/s]


Epoch Time: 0m 59s
	Train Loss: 0.678 | Train Acc: 56.15%
	 Val. Loss: 0.686 |  Val. Acc: 56.98%
Epoch: 02


...


Epoch Time: 0m 38s
	Train Loss: 0.270 | Train Acc: 88.78%
	 Val. Loss: 0.454 |  Val. Acc: 83.15%
Epoch: 10


train:   0%|          | 0/125 [00:00<?, ?it/s]

evaluate:   0%|          | 0/32 [00:00<?, ?it/s]


Epoch Time: 0m 37s
	Train Loss: 0.239 | Train Acc: 90.39%
	 Val. Loss: 0.573 |  Val. Acc: 81.84%


(0.2394829098582268, 0.903875, 0.573499342892319, 0.818359375, None, None)

# BERT

## Utils for Text Classification using Huggingface Libary

In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=-1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="weighted")
    precision = precision_score(y_true=labels, y_pred=pred, average="weighted")
    f1 = f1_score(y_true=labels, y_pred=pred, average="weighted")
    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

## Use Huggingface Library

In [18]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [19]:
def _load_dataset_for_BERT(tokenizer_name, num_samples=10000, train_test_ratio=0.8, seed=42, test_data_path=None):

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels
        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item
        def __len__(self):
            return len(self.labels)

    def preprocess(tokenizer, dataset):
        return tokenizer(dataset, truncation=True)

    dataset = load_dataset("csv", data_files=f"../data/review_{num_samples // 1000}k.csv", split="train")
    dataset = dataset.train_test_split(test_size=1 - train_test_ratio, seed=seed)
    train_dataset, eval_dataset = dataset["train"], dataset["test"]

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    train_encodings, eval_encodings, test_encodings = preprocess(tokenizer, train_dataset['review']), preprocess(tokenizer, eval_dataset['review']), None
    train_dataset, eval_dataset, test_dataset = CustomDataset(train_encodings, train_dataset['sentiment']), CustomDataset(eval_encodings, eval_dataset['sentiment']), None

    if test_data_path is not None:
        test_dataset = load_dataset("csv", data_files=test_data_path, split="train")
        test_encodings = preprocess(tokenizer, test_dataset['review'])
        test_dataset = CustomDataset(test_encodings, test_dataset['sentiment'])

    return train_dataset, eval_dataset, test_dataset

In [20]:
def run_BERT(model_name, args, num_samples=10000, seed=0, test_data_path=None, verbose=True):
    """
    You do not have to consider test_data_path, since it will be used for grading only.
    You can modify this run function for training your own model in the marked area below.
    """
    train_dataset, eval_dataset, test_dataset = _load_dataset_for_BERT(model_name, num_samples, seed=seed, test_data_path=test_data_path)

    # model
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_loss = None
    test_acc = None

    trainer.train()
    eval_result = trainer.evaluate(eval_dataset)
    eval_loss, eval_acc = eval_result['eval_loss'], eval_result['eval_accuracy']

    if test_dataset is not None: # then this will be None since test_data_path is None?
        test_result = trainer.predict(test_dataset)
        test_loss, test_acc = test_result['eval_loss'], test_result['eval_accuracy']
        print(f'\t Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

    return eval_loss, eval_acc, test_loss, test_acc

## Run BERT

In [21]:
def set_hyperparameter_for_BERT():
    """ Set your best hyperparameters for your model
    """
    args = TrainingArguments(
        output_dir='./DistilBERT/',
        learning_rate=1e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )
    return args

In [22]:
args = set_hyperparameter_for_BERT()
run_BERT('distilbert-base-cased', args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,No log,0.281477,0.889500,0.889500,0.890830,0.889333
2,0.322600,0.268304,0.905000,0.905000,0.906280,0.904979
3,0.322600,0.282958,0.900000,0.900000,0.900762,0.899903
4,0.151500,0.310899,0.901000,0.901000,0.901528,0.901003
5,0.151500,0.322533,0.900500,0.900500,0.900944,0.900505


(0.26830431818962097, 0.905, None, None)